In [1]:
import os
import re
import cv2
import numpy as np
import tensorflow as tf
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score
)
from collections import defaultdict
from tqdm import tqdm

tf.keras.backend.set_floatx('float64')

class PrototypicalEvaluator:
    def __init__(self, encoder, data_root, class_labels, num_support=8):
        self.encoder = encoder
        self.class_labels = class_labels
        self.num_support = num_support
        self.data_root = data_root
        self.class_map = {v: k for k, v in class_labels.items()}
        self.dataset = self._load_dataset()
        self.classes = list(self.dataset.keys())
        print(f"\nSuccessfully loaded {len(self.classes)} classes:")
        print("\n".join(self.classes))

    def _load_dataset(self):
        dataset = defaultdict(list)
        for class_name in self.class_labels.values():
            class_path = os.path.join(self.data_root, class_name)
            if not os.path.isdir(class_path):
                print(f"⚠️ Skipping missing class: {class_path}")
                continue
            for video_folder in os.listdir(class_path):
                video_path = os.path.join(class_path, video_folder)
                if os.path.isdir(video_path):
                    frames = self._load_video_frames(video_path)
                    if frames:
                        dataset[class_name].append(frames)
        return dataset

    def _load_video_frames(self, video_path):
        try:
            frame_files = sorted(
                [f for f in os.listdir(video_path) if f.lower().startswith('frame') and f.lower().endswith('.jpg')],
                key=lambda x: int(re.search(r'\d+', x).group())
            )
        except Exception as e:
            print(f"Error sorting frames: {str(e)}")
            return []
        video_frames = []
        for frame_file in frame_files:
            try:
                image = cv2.imread(os.path.join(video_path, frame_file), cv2.IMREAD_GRAYSCALE)
                if image is None:
                    raise ValueError("Failed to read image")
                image = image.astype(np.float64)
                resized = tf.image.resize(image[..., None], (28, 28)).numpy().squeeze()
                resized = (255.0 - resized) / 255.0  # Normalize to [0,1]
                video_frames.append(resized)
            except Exception as e:
                print(f"Skipping {frame_file}: {str(e)}")
        return video_frames

    def evaluate(self, num_episodes=100, n_way=8):
        """Evaluate model performance with full class coverage using single frame queries"""
        if n_way != len(self.class_labels):
            raise ValueError(f"n_way must match number of classes ({len(self.class_labels)})")
        for cls in self.classes:
            if len(self.dataset[cls]) < self.num_support + 1:
                raise ValueError(f"Class {cls} has only {len(self.dataset[cls])} videos. Need at least {self.num_support + 1}")

        # Initialize metrics storage
        frame_metrics = {'true': [], 'pred': [], 'confidence': [], 'distances': []}

        # Create balanced episodes with all classes
        for _ in tqdm(range(num_episodes)):
            support_set, query_set = self._create_episode(self.classes)
            prototypes = self._create_prototypes(support_set)

            for video, true_class in query_set:
                # Select a random frame from the video as query
                if len(video) > 0:
                    query_frame = video[np.random.randint(0, len(video))]
                    pred_result = self._predict_frame(query_frame, prototypes)
                    frame_metrics['true'].append(true_class)
                    frame_metrics['pred'].append(pred_result['class'])
                    frame_metrics['confidence'].append(pred_result['confidence'])
                    frame_metrics['distances'].append(pred_result['distances'])

        return self._compile_results(frame_metrics)

    def _create_episode(self, selected_classes):
        support_set = defaultdict(list)
        query_set = []
        for cls in selected_classes:
            videos = self.dataset[cls]
            np.random.shuffle(videos)
            support_set[cls] = videos[:self.num_support]
            for video in videos[self.num_support:]:
                query_set.append((video, cls))
        return support_set, query_set

    def _create_prototypes(self, support_set):
        """Create class prototypes by max-pooling frame embeddings and averaging across support videos"""
        prototypes = {}
        for cls, videos in support_set.items():
            class_video_embeddings = []
            for video in videos:
                video_tensor = tf.convert_to_tensor(np.array(video, dtype=np.float64)[..., None])
                embeddings = self.encoder.predict(video_tensor)
                if embeddings.ndim == 1:
                    embeddings = np.expand_dims(embeddings, 0)
                # Max-pool across frames
                video_emb = np.max(embeddings, axis=0)
                class_video_embeddings.append(video_emb)
            # Prototype is mean of support video embeddings
            prototypes[cls] = np.mean(class_video_embeddings, axis=0)
        return prototypes

    def _predict_frame(self, frame, prototypes):
        """Predict class for a single frame by comparing to class prototypes"""
        frame_tensor = tf.convert_to_tensor(np.array([frame], dtype=np.float64)[..., None])
        embedding = self.encoder.predict(frame_tensor)
        if embedding.ndim == 1:
            embedding = embedding.reshape(1, -1)
        frame_emb = embedding[0]  # Get the embedding for the single frame

        # Compute distances to each class prototype
        distances = {cls: np.linalg.norm(frame_emb - proto) for cls, proto in prototypes.items()}
        predicted_class = min(distances, key=distances.get)
        confidence = 1 - (distances[predicted_class] / sum(distances.values()))

        return {
            'class': predicted_class, 
            'confidence': confidence,
            'distances': distances
        }

    def _compile_results(self, frame_metrics):
        """Compute accuracy, precision, recall, f1 and average confidence"""
        true = frame_metrics['true']
        pred = frame_metrics['pred']

        results = {}
        results['frame_accuracy'] = np.mean(np.array(true) == np.array(pred))
        # Macro-averaged precision, recall, f1
        results['precision'] = precision_score(true, pred, average='macro', zero_division=0)
        results['recall'] = recall_score(true, pred, average='macro', zero_division=0)
        results['f1_score'] = f1_score(true, pred, average='macro', zero_division=0)
        results['avg_confidence'] = np.mean(frame_metrics['confidence'])
        results['distances'] = frame_metrics['distances']

        return results

# Usage example
if __name__ == "__main__":
    encoder = tf.keras.models.load_model(
        r"C:\Users\Mehdi\Desktop\work\SEMESTRE 4\P2M\CODE\SN_fsl\SN_fsl\data_saved\prototypical_net",
        compile=False
    )
    if isinstance(encoder, tf.keras.Sequential):
        encoder = tf.keras.Model(inputs=encoder.inputs, outputs=encoder.outputs)

    inputs = tf.keras.Input(shape=(28, 28, 1), dtype=tf.float64)
    x = tf.keras.layers.Lambda(lambda x: tf.cast(x, tf.float64))(inputs)
    outputs = encoder(x)
    encoder = tf.keras.Model(inputs, outputs)

    CLASS_LABELS = {
        0: 'Carcinoma',
        1: 'Extreme_polipoid',
        2: 'Laryngitis',
        3: 'leukoplacia',
        4: 'papilloma',
        5: 'scar',
        6: 'vocal_fold_cyst',
        7: 'Vocal_insufficiency',
    }
    evaluator = PrototypicalEvaluator(
        encoder=encoder,
        data_root=r"C:\Users\Mehdi\Desktop\work\SEMESTRE 4\P2M\data_frames\max_frames_fsl",
        class_labels=CLASS_LABELS,
        num_support=1
    )
    results = evaluator.evaluate(num_episodes=100)
    print(f"Frame-level Accuracy: {results['frame_accuracy']:.2%}")
    print(f"Precision (macro): {results['precision']:.2%}")
    print(f"Recall (macro): {results['recall']:.2%}")
    print(f"F1-score (macro): {results['f1_score']:.2%}")
    print(f"Average Confidence: {results['avg_confidence']:.2%}")
    
    # Print distances for the first few queries
    print("\nSample distances to each class prototype:")
    for i, dist_dict in enumerate(results['distances'][:5]):
        print(f"\nQuery {i+1}:")
        for cls, dist in sorted(dist_dict.items(), key=lambda x: x[1]):
            print(f"  {cls}: {dist:.4f}")



Successfully loaded 8 classes:
Carcinoma
Extreme_polipoid
Laryngitis
leukoplacia
papilloma
scar
vocal_fold_cyst
Vocal_insufficiency


  0%|          | 0/100 [00:00<?, ?it/s]

1/1 [==============================] - 0s 18ms/step


  1%|          | 1/100 [00:09<15:07,  9.17s/it]

1/1 [==============================] - 0s 19ms/step


...

1/1 [==============================] - 0s 20ms/step


 99%|█████████▉| 99/100 [01:59<00:01,  1.10s/it]

1/1 [==============================] - 0s 19ms/step


100%|██████████| 100/100 [02:00<00:00,  1.20s/it]

Frame-level Accuracy: 39.15%
Precision (macro): 42.12%
Recall (macro): 50.53%
F1-score (macro): 41.45%
Average Confidence: 94.19%

Sample distances to each class prototype:

Query 1:
  Carcinoma: 30.6926
  Laryngitis: 69.8523
  Vocal_insufficiency: 71.7355
  papilloma: 82.2965
  scar: 98.7242
  Extreme_polipoid: 100.1532
  vocal_fold_cyst: 107.5108
  leukoplacia: 179.1025

Query 2:
  scar: 54.7668
  Carcinoma: 55.2546
  Vocal_insufficiency: 68.1865
  Laryngitis: 73.2242
  Extreme_polipoid: 86.5398
  papilloma: 103.5758
  vocal_fold_cyst: 114.8860
  leukoplacia: 178.1703

Query 3:
  Vocal_insufficiency: 88.1023
  scar: 112.6448
  vocal_fold_cyst: 120.0259
  Carcinoma: 148.2629
  papilloma: 150.7821
  Extreme_polipoid: 161.8651
  Laryngitis: 161.9180
  leukoplacia: 234.9709

Query 4:
  leukoplacia: 2.0442
  Carcinoma: 178.7517
  Laryngitis: 184.7585
  scar: 188.5101
  Vocal_insufficiency: 189.4187
  Extreme_polipoid: 202.2728
  papilloma: 206.3253
  vocal_fold_cyst: 214.7074

Query 5:
  